# Variants-Experiment 1.1

## Concatenated hidden representations.
## Mode. Compute classifications of all tiny networks, in all HiCo layers, and take the mode as the final classification.


In [1]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten
from keras.datasets import mnist
from keras.utils import to_categorical
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import random
from scipy import stats
from sklearn.metrics import accuracy_score
from keras import backend as K

import os
import pickle
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
SUB_REGION_SCALE = 14
NUM_HICO_LAYER = 5
NUM_TN = [10, 8, 6, 4, 2]
NUM_CONNECTION = [0, 5, 4, 3, 2]

#dataset specific parameters
NUM_CLASS = 10

## Data Pre-Processing

In [3]:
with open('/content/drive/My Drive/fyp/MNIST.pickle', 'rb') as f:
    X_train_cropped_list, y_train_one_hot, X_test_cropped_list, y_test_one_hot, coordinate_list, scale_list = pickle.load(f)

## HiCo Layer 1

In [4]:
#build ANN model
ensemble = []
for i in range(NUM_TN[0]):
  model = Sequential()
  model.add(Dense(64, activation='relu', input_dim=SUB_REGION_SCALE*SUB_REGION_SCALE))
  model.add(Dense(64, activation='relu'))
  model.add(Dense(NUM_CLASS, activation = 'softmax'))
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
  ensemble.append((model, i, coordinate_list[i], scale_list[i]))

In [5]:
for i in range(NUM_TN[0]):
  ensemble[i][0].load_weights('/content/drive/My Drive/fyp/MNIST_' + str(i) + '.h5')

In [6]:
hico_layers = []
hico_layers.append(ensemble)

In [7]:
del hico_layers[1:]

## HiCo Layer 2+

In [8]:
# Function to get input of layer i-1
def get_previous_layer_input(hico_layers, layer, tn, train_image):
  input = []
    
  if layer == 0:
    previous_layer_input = train_image[tn[1]]
    return previous_layer_input

  elif layer > 0:
    for i in range(NUM_CONNECTION[layer]):
      get_input = (K.function(hico_layers[layer-1][tn[1][i]][0].layers[0].input, hico_layers[layer-1][tn[1][i]][0].layers[1].output))
      input.append(get_input(get_previous_layer_input(hico_layers, layer-1, hico_layers[layer-1][tn[1][i]], train_image)))
      
    input = np.array(input)
    input = np.concatenate(input, axis=1)
    previous_layer_input = input
    return previous_layer_input

In [9]:
for i in range(1, NUM_HICO_LAYER):
  print('Layer %d' %i)
  ensemble = []
  X_train_input_list = []
  X_test_input_list = []

  for j in range(NUM_TN[i]):
    # Build model of HiCo layer i
    connection = tuple(random.sample(range(len(hico_layers[i-1])), k=NUM_CONNECTION[i]))
    model = Sequential()
    model.add(Dense(64, activation='relu', input_dim=64*NUM_CONNECTION[i]))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(NUM_CLASS, activation = 'softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    ensemble.append((model, connection))
  hico_layers.append(ensemble)
  print('HICO LENGTH')
  print(len(hico_layers))

  for j in range(NUM_TN[i]):
    # Get train hidden representation from HiCo layer i-1
    X_train_input = []
    for k in range(NUM_CONNECTION[i]):
      get_input = (K.function(hico_layers[i-1][hico_layers[i][j][1][k]][0].layers[0].input, hico_layers[i-1][hico_layers[i][j][1][k]][0].layers[1].output))
      X_train_input.append(get_input(get_previous_layer_input(hico_layers, i-1, hico_layers[i-1][hico_layers[i][j][1][k]], X_train_cropped_list)))
    X_train_input = np.array(X_train_input)
    X_train_input = np.concatenate(X_train_input, axis=1)
    X_train_input_list.append(X_train_input)

    # Get test hidden representation from HiCo layer i-1
    X_test_input = []
    for k in range(NUM_CONNECTION[i]):
      get_input = (K.function(hico_layers[i-1][hico_layers[i][j][1][k]][0].layers[0].input, hico_layers[i-1][hico_layers[i][j][1][k]][0].layers[1].output))
      X_test_input.append(get_input(get_previous_layer_input(hico_layers, i-1, hico_layers[i-1][hico_layers[i][j][1][k]], X_test_cropped_list)))
    X_test_input = np.array(X_test_input)
    X_test_input = np.concatenate(X_test_input, axis=1)
    X_test_input_list.append(X_test_input)

  #train model of HiCo layer i
  for j in range(NUM_TN[i]):
    print('Model %d' %j)
    hico_layers[i][j][0].fit(X_train_input_list[j], y_train_one_hot, validation_data=(X_test_input_list[j], y_test_one_hot), epochs=5, batch_size=128)

Layer 1
HICO LENGTH
2
Model 0
Epoch 1/5
469/469 [==============================] - 2s 3ms/step - loss: 0.3877 - accuracy: 0.8952 - val_loss: 0.0859 - val_accuracy: 0.9737
Epoch 2/5
469/469 [==============================] - 1s 2ms/step - loss: 0.0775 - accuracy: 0.9754 - val_loss: 0.0725 - val_accuracy: 0.9774
Epoch 3/5
469/469 [==============================] - 1s 2ms/step - loss: 0.0677 - accuracy: 0.9775 - val_loss: 0.0746 - val_accuracy: 0.9769
Epoch 4/5
469/469 [==============================] - 1s 2ms/step - loss: 0.0628 - accuracy: 0.9796 - val_loss: 0.0813 - val_accuracy: 0.9743
Epoch 5/5
469/469 [==============================] - 1s 2ms/step - loss: 0.0584 - accuracy: 0.9816 - val_loss: 0.0774 - val_accuracy: 0.9771
Model 1
Epoch 1/5
469/469 [==============================] - 1s 2ms/step - loss: 0.3613 - accuracy: 0.8989 - val_loss: 0.1017 - val_accuracy: 0.9697
Epoch 2/5
469/469 [==============================] - 1s 2ms/step - loss: 0.0729 - accuracy: 0.9768 - val_loss: 0.080

In [10]:
hico_layers[3][0][0].summary()

Model: "sequential_24"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_72 (Dense)             (None, 64)                12352     
_________________________________________________________________
dense_73 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_74 (Dense)             (None, 10)                650       
Total params: 17,162
Trainable params: 17,162
Non-trainable params: 0
_________________________________________________________________


## Model Evaluation

In [11]:
# Predict on test image
y_pred_list = []
for j in range(NUM_TN[0]):
  y_pred = np.argmax(hico_layers[0][j][0].predict(X_test_cropped_list[j]), axis=1)
  y_pred_list.append(y_pred)

for i in range(1, NUM_HICO_LAYER):
  for j in range(NUM_TN[i]):

    # Get test hidden representation from HiCo layer i-1
    X_test_input = []
    for k in range(NUM_CONNECTION[i]):
      get_input = (K.function(hico_layers[i-1][hico_layers[i][j][1][k]][0].layers[0].input, hico_layers[i-1][hico_layers[i][j][1][k]][0].layers[1].output))
      X_test_input.append(get_input(get_previous_layer_input(hico_layers, i-1, hico_layers[i-1][hico_layers[i][j][1][k]], X_test_cropped_list)))
    X_test_input = np.array(X_test_input)
    X_test_input = np.concatenate(X_test_input, axis=1)
    y_pred = np.argmax(hico_layers[i][j][0].predict(X_test_input), axis=1)
    y_pred_list.append(y_pred)

# HiCo voting (mode)
y_pred_list = np.array(y_pred_list)
y_pred_list = np.transpose(y_pred_list, (1, 0))
y_pred_list = stats.mode(y_pred_list, axis=1)[0]
y_pred_list = np.squeeze(y_pred_list)

In [12]:
y_test = np.argmax(y_test_one_hot, axis=1)

In [13]:
accuracy = accuracy_score(y_test, y_pred_list)
accuracy

0.9853